一、函数

In [42]:
import os
import nltk
import re
import numpy as np
import pandas as pd
from collections import Counter

import sklearn
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVR, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from func import *

二、POS Sequence Pattern 挖掘

In [ ]:
tagList = [
    'NN', 'CC', 'LS', 'PDT', 'POS', 'SYM', 'NNS', 'NNP', 'NNPS', 'FW', 'CD',
    'JJ', 'JJR', 'JJS', 'IN', 'TO', 'DT', 'EX', 'PRP', 'PRP$', 'WDT', 'WP',
    'WP$', 'MD', 'VB', 'VBZ', 'VBP', 'VBD', 'VBN', 'VBG', 'RB', 'RBR', 'RBS',
    'RP', 'WRB', 'UH', '.'
]

for gender in [0, 1]:
    if gender == 0:
        txtDir = './postprocess_blogs/blogs/female/'
    else:
        txtDir = './postprocess_blogs/blogs/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in range(0, len(blogs_gender)):
        m = blogs_gender[i]
        print("Processing: ", m)
        print("Files:", len(os.listdir(txtDir + m)))
        for file in os.listdir(txtDir + m):
            text = gettext(txtDir + m + '/' + file)
            sentences = nltk.sent_tokenize(text)
            CorpusPOS(sentences)

infile = open('CorpusPOS.txt', 'r')
cPOS = infile.readlines()
infile.close()
(a, b, c, d, e) = calc_probabilities(cPOS)
q1_output(a, b, c, d, e)

Prob = {}
infile = open('probabilities.txt', 'r')
prob_text = infile.readlines()

for sentence in prob_text:
    keyValPair = sentence.split(":")
    Prob[keyValPair[0]] = float(keyValPair[1][:-1])

infile.close()

posFeatures = minePOSPats(cPOS)

三、 feature engineering

In [ ]:
names = []
F_features = []
GRF_features = []
WC_features = []
POS_features = []
labels = []

for gender in [0, 1]:
    if gender == 0:
        txtDir = './postprocess_blogs/blogs/female/'
    else:
        txtDir = './postprocess_blogs/blogs/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in range(0, len(blogs_gender)):
        m = blogs_gender[i]
        print("Processing: ", m)
        print("Files:", len(os.listdir(txtDir + m)))
        for file in os.listdir(txtDir + m):
            name = txtDir + m + '/' + file
            text = gettext(name)
            words = getwords(text)
            sentences = nltk.sent_tokenize(text)
            words_s = split(words)
            tags = [nltk.pos_tag(word) for word in words]
            tags_s = split(tags)
            words_l = wordlemmatize(tags_s)

            F_feature = F_measure(tags_s)
            GRF_feature = Gender_Preferential_Features(words_l)
            WC_feature = Word_Classes_Feature(words_l)

            textTags = ""
            for word, tag in tags_s:
                if tag in tagList:
                    textTags = textTags + tag + " "

            POS_feature = []

            for feature in posFeatures:
                if feature in textTags:
                    POS_feature.append(1)
                else:
                    POS_feature.append(0)

            names.append(name)
            F_features.append(F_feature)
            GRF_features.append(GRF_feature)
            WC_features.append(WC_feature)
            POS_features.append(POS_feature)
            labels.append(gender)

In [40]:
map1 = {'name': names, 'label': labels, 'F_feature': F_features}

for i in range(23):
    key = 'WC_' + str(i + 1)
    value = WC_features_l[i]
    map1[key] = value

for i in range(10):
    key = 'GRF_' + str(i + 1)
    value = GRF_features_l[i]
    map1[key] = value

for i in range(33):
    key = 'POS_' + str(i + 1)
    value = POS_features_l[i]
    map1[key] = value

allofall = pd.DataFrame(map1)
F_features_u = np.array(F_features)
F_features_u = (F_features_u - np.mean(F_features_u)) / np.std(F_features_u)
allofall['F_feature'] = F_features_u

In [4]:
#allofall.to_csv('allofall.csv',index = False)
allofall = pd.read_csv('allofall.csv')
df_per_txt = pd.read_csv('blogs_genderbias.csv')

In [ ]:
allofall['word ratio'], allofall['bias'] = df_per_txt['word ratio'], df_per_txt['bias']
allofall

In [34]:
data, target = allofall.drop(
    columns=['name', 'label']).iloc[:].values, allofall['label'].values
train_X, test_X, train_y, test_y = train_test_split(data,
                                                    target,
                                                    test_size=0.3,
                                                    random_state=9,
                                                    shuffle=True)
data, target = pd.concat(
    (pd.DataFrame(train_X), pd.DataFrame(test_X))), pd.concat(
        (pd.DataFrame(train_y), pd.DataFrame(test_y)))
data1 = data.iloc[:, -2:]

In [36]:
data2 = data.iloc[:, :-2]

In [37]:
from sklearn.model_selection import cross_val_score

rnd_clf = RandomForestClassifier(n_estimators=800,
                                 max_leaf_nodes=50,
                                 n_jobs=-1)
scores_rnd_clf_cv = cross_val_score(rnd_clf,
                                    data2.values,
                                    target.values.reshape(-1, ),
                                    cv=5)
print(scores_rnd_clf_cv)
print("Accuracy: %0.2f (+/- %0.2f)" %
      (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))

[0.751 0.742 0.774 0.758 0.756]
Accuracy: 0.76 (+/- 0.02)


In [32]:
rnd_clf = RandomForestClassifier(n_estimators=800,
                                 max_leaf_nodes=50,
                                 n_jobs=-1)
rnd_clf.fit(train_X, train_y)
y_pred = rnd_clf.predict(test_X)
accuracy_score(test_y, y_pred)

0.7866666666666666

In [58]:
from sklearn.model_selection import cross_val_score

rnd_clf = RandomForestClassifier(n_estimators=800,
                                 max_leaf_nodes=50,
                                 n_jobs=-1)
scores_rnd_clf_cv = cross_val_score(rnd_clf,data1.values,target1.values.reshape(-1,),cv=5)
print(scores_rnd_clf_cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))

[0.7025 0.705  0.7225 0.755  0.7525]
Accuracy: 0.73 (+/- 0.05)


In [7]:
from sklearn.model_selection import cross_val_score

rnd_clf = RandomForestClassifier(n_estimators=1000,
                                 max_leaf_nodes=50,
                                 n_jobs=-1)
scores_rnd_clf_cv = cross_val_score(rnd_clf,data.values,target.values.reshape(-1,),cv=5)
print(scores_rnd_clf_cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))

[0.762 0.794 0.788 0.78  0.807]
Accuracy: 0.79 (+/- 0.03)


In [9]:
rnd_clf = LogisticRegression(solver="liblinear", random_state=42)
scores_rnd_clf_cv = cross_val_score(rnd_clf,data.values,target.values.reshape(-1,),cv=5)
print(scores_rnd_clf_cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))

[0.736 0.771 0.755 0.744 0.754]
Accuracy: 0.75 (+/- 0.02)


In [ ]:
rnd_clf = LogisticRegression(solver="liblinear", random_state=42)
scores_rnd_clf_cv = cross_val_score(rnd_clf,data1.values,target.values.reshape(-1,),cv=5)
print(scores_rnd_clf_cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))

In [64]:
rnd_clf = SVC(kernel='rbf', gamma=0.1, probability=True, C=1000)
scores_rnd_clf_cv = cross_val_score(rnd_clf,data1.values,target.values.reshape(-1,),cv=5)
print(scores_rnd_clf_cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))

[0.685 0.735 0.72  0.735 0.7  ]
Accuracy: 0.71 (+/- 0.04)


In [10]:
rnd_clf = SVC(kernel='rbf', gamma=0.1, probability=True, C=1000)
scores_rnd_clf_cv = cross_val_score(rnd_clf,data.values,target.values.reshape(-1,),cv=5)
print(scores_rnd_clf_cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))

[0.773 0.803 0.782 0.781 0.787]
Accuracy: 0.79 (+/- 0.02)


In [66]:
rnd_clf = sklearn.tree.DecisionTreeClassifier(max_leaf_nodes=50)
scores_rnd_clf_cv = cross_val_score(rnd_clf,data.values,target.values.reshape(-1,),cv=5)
print(scores_rnd_clf_cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))

[0.69   0.715  0.7025 0.755  0.7125]
Accuracy: 0.71 (+/- 0.04)


In [22]:
y_pred = rnd_clf.predict(test_X)
accuracy_score(test_y, y_pred)

0.775

In [ ]:
param_grid = [
    {
        'kernel': ['poly'],
        'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0],
        'degree': [2, 3, 4, 5, 6]
    },
    {
        'kernel': ['rbf'],
        'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0],
        'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]
    },
]

svc_clf_gs = SVC()
grid_search = GridSearchCV(svc_clf_gs,
                           param_grid,
                           cv=10,
                           scoring='accuracy',
                           verbose=2,
                           n_jobs=-1)
grid_search.fit(data, target)

In [ ]:
cvres = grid_search.cv_results_
for score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(score, params)

In [ ]:
import keras
import keras.backend as K
from keras import models, layers, regularizers, optimizers

K.clear_session()

model = models.Sequential()
model.add(layers.Dense(8, activation='relu', input_shape=(65,),))
model.add(layers.Dense(1, activation='sigmoid',))
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
                loss='binary_crossentropy',
                metrics=['accuracy'])
model.summary()

In [ ]:
callbacks_list = [
                keras.callbacks.EarlyStopping(
                monitor='val_acc',
                patience=10,
                ),
keras.callbacks.ModelCheckpoint(
                filepath='my_model.h5',
                monitor='val_loss',
                save_best_only=True,
                )
]

history = model.fit(data, target, batch_size=128, epochs=100, verbose = 1, 
                    validation_split=0.1, callbacks=callbacks_list, shuffle=True)

In [3]:
df_per_txt = pd.read_csv('blogs_genderbias.csv')
df_per_txt

,bias,gender,word ratio
0,0.289545,0.0,5.000000e-01
1,0.000000,0.0,1.000000e+00
2,0.940899,0.0,6.502463e-01
3,0.810930,0.0,4.347826e-01
4,0.133531,0.0,5.000000e-01
5,0.541787,0.0,5.824561e-01
6,1.165503,0.0,7.419355e-01
7,0.000000,0.0,7.500001e-01
8,0.810930,0.0,6.000000e-01
9,0.154151,0.0,5.500000e-01


In [25]:
allofall.to_csv('blogs_features.csv')

In [7]:
tagList = [
    'NN', 'CC', 'LS', 'PDT', 'POS', 'SYM', 'NNS', 'NNP', 'NNPS', 'FW', 'CD',
    'JJ', 'JJR', 'JJS', 'IN', 'TO', 'DT', 'EX', 'PRP', 'PRP$', 'WDT', 'WP',
    'WP$', 'MD', 'VB', 'VBZ', 'VBP', 'VBD', 'VBN', 'VBG', 'RB', 'RBR', 'RBS',
    'RP', 'WRB', 'UH', '.'
]

for gender in [0, 1]:
    if gender == 0:
        txtDir = './2000novel/female/'
    else:
        txtDir = './2000novel/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    print("Files:", len(blogs_gender))
    for m in blogs_gender:
        text = gettext(txtDir + m)
        sentences = nltk.sent_tokenize(text)
        CorpusPOS(sentences)


infile = open('CorpusPOS.txt', 'r')
cPOS = infile.readlines()
infile.close()

cPOS_new = []
for sent in cPOS:
    sent_new = re.sub(r'[^\w\s]','',sent)
    cPOS_new.append(sent_new)

(a, b, c, d, e, f, g) = calc_probabilities(cPOS_new)
q1_output(a, b, c, d, e, f, g)

Prob = {}
infile = open('probabilities.txt', 'r')
prob_text = infile.readlines()

for sentence in prob_text:
    keyValPair = sentence.split(":")
    Prob[keyValPair[0]] = float(keyValPair[1][:-1])

infile.close()

posFeatures = minePOSPats(cPOS_new)

In [8]:
posFeatures

['NN',
 'CC',
 'NNS',
 'NNP',
 'IN',
 'TO',
 'DT',
 'PRP',
 'VB',
 'VBD',
 'RB',
 'RP',
 'JJ',
 'VBN']

In [3]:
names = []
F_features = []
GRF_features = []
WC_features = []
POS_features = []
labels = []

for gender in [0, 1]:
    if gender == 0:
        txtDir = './5000/female/'
    else:
        txtDir = './5000/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    print("Files:", len(blogs_gender))
    for m in blogs_gender:
        name = txtDir + m
        text = gettext(name)
        words = nltk.word_tokenize(text)
        sentences = nltk.sent_tokenize(text)
        tags = nltk.pos_tag(words)
        words_l = wordlemmatize(tags)

        F_feature = F_measure(tags)
        GRF_feature = Gender_Preferential_Features(words_l)
        WC_feature = Word_Classes_Feature(words_l)

        #textTags = ""
        #for word, tag in tags_s:
        #    if tag in tagList:
        #        textTags = textTags + tag + " "

        #POS_feature = []

        #for feature in posFeatures:
        #    if feature in textTags:
        #        POS_feature.append(1)
        #    else:
        #        POS_feature.append(0)
        names.append(name)
        F_features.append(F_feature)
        GRF_features.append(GRF_feature)
        WC_features.append(WC_feature)
        #POS_features.append(POS_feature)
        labels.append(gender)
        

Processing gender: ./5000/female/
Files: 2500
Processing gender: ./5000/male/
Files: 2500


In [4]:
def getsingle(features, n):
    single = []
    for item in features:
        single.append(item[n])
    return single


WC_features_l = []
for i in range(len(WC_features[0])):
    n = i
    WC_features_l.append(getsingle(WC_features, n))

GRF_features_l = []
for i in range(len(GRF_features[0])):
    n = i
    GRF_features_l.append(getsingle(GRF_features, n))

#POS_features_l = []
#for i in range(len(POS_features[0])):
#    n = i
#    POS_features_l.append(getsingle(POS_features, n))

In [6]:
map1 = {'name': names, 'label': labels, 'F_feature': F_features}

for i in range(len(WC_features[0])):
    key = 'WC_' + str(i + 1)
    value = WC_features_l[i]
    map1[key] = value

for i in range(len(GRF_features[0])):
    key = 'GRF_' + str(i + 1)
    value = GRF_features_l[i]
    map1[key] = value

#for i in range(len(POS_features[0])):
#    key = 'POS_' + str(i + 1)
#    value = POS_features_l[i]
#    map1[key] = value

allofall = pd.DataFrame(map1)

F_features_u = np.array(F_features)
F_features_u = (F_features_u - np.mean(F_features_u)) / np.std(F_features_u)
allofall['F_feature'] = F_features_u

allofall.to_csv('allofall_5000.csv',index = False)
#allofall = pd.read_csv('allofall.csv')
#df_per_txt = pd.read_csv('blogs_genderbias.csv')

In [8]:
df_per_txt = pd.read_csv('5000_genderbias.csv')

In [9]:
allofall['bias'], allofall['word ratio'] = df_per_txt['bias'], df_per_txt['word ratio']

In [49]:
allofall.to_csv('5000_features.csv',index=False)
#allofall = pd.read_csv('5000_features.csv')

In [44]:
lengths = []
for gender in [0, 1]:
    if gender == 0:
        txtDir = './5000/female/'
    else:
        txtDir = './5000/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    print("Files:", len(blogs_gender))
    for m in blogs_gender:
        text = gettext(txtDir + m)
        words = nltk.word_tokenize(text)
        length = len(words)
        
        lengths.append(length)

Processing gender: ./5000/female/
Files: 2500
Processing gender: ./5000/male/
Files: 2500


In [45]:
allofall = pd.read_csv('5000_features.csv')

In [47]:
allofall['word count'] = lengths

In [51]:
allofall['word count'].mean()

72007.5714